In [1]:
import pandas as pd
from datasets import load_from_disk

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)

from src.data.utilities import extract_slots, remove_empty_slots

In [2]:
df = pd.concat(
    [
        load_from_disk("../data/prepared/base/sgd_ops_nohist+prev")[split].to_pandas().assign(split=split)
        for split in ("train", "validation", "test")
    ]
)

In [7]:
df["states"] = df["states"].map(remove_empty_slots)
df["previous_states"] = df["previous_states"].map(remove_empty_slots)

In [10]:
df.drop(columns=["schema"])

,dialogue_id,turn_id,sys_utt,usr_utt,states,split,previous_states,previous_id,sys_history,usr_history,input_text,target_text,input_ids,attention_mask,labels
0,100_00000,0,none,I would like to find a concert to attend in SF.,"{'events_1-category': 'Music', 'events_1-city_of_event': 'SF'}",train,None,None,[],[],generate update operations: dialogue: system: none user: I would like to find a concert to attend in SF. <sep> previous dialogue states: none <sep> operations:,INSERT events_1-category = Music ; INSERT events_1-city_of_event = SF,"[3806, 2270, 2673, 10, 7478, 10, 358, 10, 5839, 1139, 10, 27, 133, 114, 12, 253, 3, 9, 4219, 12, 2467, 16, 3, 7016, 5, 3, 32100, 1767, 7478, 2315, 10, 5839, 3, 32100, 2673, 10, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[3, 14750, 24203, 984, 834, 536, 18, 8367, 839, 651, 3274, 3057, 3, 117, 3, 14750, 24203, 984, 834, 536, 18, 6726, 834, 858, 834, 15, 2169, 3274, 3, 7016, 1]"
1,100_00000,1,Allan Rayman is performing at August Hall on March 9th at 6 pm. It is a very popular event.,Can you tell me more about it? Where is the venue located?,"{'events_1-category': 'Music', 'events_1-city_of_event': 'SF'}",train,"{'events_1-category': 'Music', 'events_1-city_of_event': 'SF'}",100_00000,[none],[I would like to find a concert to attend in SF.],generate update operations: dialogue: system: Allan Rayman is performing at August Hall on March 9th at 6 pm. It is a very popular event. user: Can you tell me more about it? Where is the venue located? <sep> previous dialogue states: events_1-category = Music; events_1-city_of_event = SF <sep> operations:,none,"[3806, 2270, 2673, 10, 7478, 10, 358, 10, 432, 152, 8279, 348, 19, 5505, 44, 1660, 2501, 30, 1332, 668, 189, 44, 431, 6366, 5, 94, 19, 3, 9, 182, 1012, 605, 5, 1139, 10, 1072, 25, 817, 140, 72, 81, 34, 58, 2840, 19, 8, 5669, 1069, 58, 3, 32100, 1767, 7478, 2315, 10, 984, 834, 536, 18, 8367, 839, 651, 3274, 3057, 117, 984, 834, 536, 18, 6726, 834, 858, 834, 15, 2169, 3274, 3, 7016, 3, 32100, 2673, 10, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[5839, 1]"
2,100_00000,2,It is a Hip Hop concert at 420 Mason Street.,That sounds fun.,"{'events_1-category': 'Music', 'events_1-city_of_event': 'SF', 'events_1-date': 'March 9th', 'events_1-event_name': 'Allan Rayman'}",train,"{'events_1-category': 'Music', 'events_1-city_of_event': 'SF'}",100_00000,"[none, Allan Rayman is performing at August Hall on March 9th at 6 pm. It is a very popular event.]","[I would like to find a concert to attend in SF., Can you tell me more about it? Where is the venue located?]",generate update operations: dialogue: system: It is a Hip Hop concert at 420 Mason Street. user: That sounds fun. <sep> previous dialogue states: events_1-city_of_event = SF; events_1-category = Music <sep> operations:,INSERT events_1-date = March 9th ; INSERT events_1-event_name = Allan Rayman,"[3806, 2270, 2673, 10, 7478, 10, 358, 10, 94, 19, 3, 9, 16337, 15139, 4219, 44, 3, 21899, 15924, 1887, 5, 1139, 10, 466, 2993, 694, 5, 3, 32100, 1767, 7478, 2315, 10, 984, 834, 536, 18, 6726, 834, 858, 834, 15, 2169, 3274, 3, 7016, 117, 984, 834, 536, 18, 8367, 839, 651, 3274, 3057, 3, 32100, 2673, 10, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[3, 14750, 24203, 984, 834, 536, 18, 5522, 3274, 1332, 668, 189, 3, 117, 3, 14750, 24203, 984, 834, 536, 18, 15, 2169, 834, 4350, 3274, 432, 152, 8279, 348, 1]"
3,100_00000,3,Would you like to purchase tickets?,Not yet. Can you help me find a movie in the same area?,"{'events_1-category': 'Music', 'events_1-city_of_event': 'SF', 'events_1-date': 'Mar

In [30]:
list_dfs = []
for v in ("21", "22", "23", "24"):
    ds = load_from_disk(f"../data/processed/multiwoz_{v}")
    df = pd.concat([ds[split].to_pandas().assign(split=split) for split in ds]).assign(version=v)
    list_dfs.append(df)

In [31]:
df = pd.concat(list_dfs)

In [32]:
df["states"] = df["states"].map(remove_empty_slots)
df["slots"] = df["states"].map(extract_slots)

In [45]:
stats = (
    df.groupby(["dialogue_id", "split", "version"])
    .agg(
        n_domains=("turn_services", lambda ex: len(set(i for j in ex for i in j))),
        n_turns=("states", len),
        n_slots=("slots", lambda ex: len(set(i for j in ex for i in j))),
    )
    .groupby(["split", "version"])
    .describe()
    .round(2)
    .T.reindex(columns=["train", "validation", "test"], level=0)
)

In [65]:
stats = (
    df.explode("slots")
    .groupby(["slots", "version"])
    .agg(
        n_dialogues=("dialogue_id", "nunique"),
        n_turns=("turn_id", len),
    )
    .unstack(-1)
)

In [82]:
stats = (
    df.explode("turn_services")
    .groupby(["turn_services", "version"])
    .agg(
        n_dialogues=("dialogue_id", "nunique"),
        n_turns=("turn_id", len),
    )
    .T.stack(0)
)

In [3]:
df = pd.concat(
    [
        load_from_disk("../data/prepared/base/mwoz22_ops_nohist+prev")[split].to_pandas().assign(split=split)
        for split in ("train", "validation", "test")
    ]
)

In [14]:
print(
    df.loc[
        df["dialogue_id"] == "MUL0003.json",
        ["dialogue_id", "turn_id", "sys_utt", "usr_utt", "states", "input_text", "target_text"],
    ]
    .assign(states=lambda df_: df_["states"].map(remove_empty_slots))
    .rename(
        columns={
            "dialogue_id": "ID",
            "turn_id": "Turn",
            "sys_utt": "System",
            "usr_utt": "User",
            "input_text": "Input",
            "target_text": "Target",
            "states": "States",
        }
    )
    .drop(columns=["ID"])
    .head(3)
    .to_latex(index=False)
)

\begin{tabular}{rlllll}
\toprule
 Turn &                                                                                                     System &                                                                                                                                                User &                                                                                                                                                                    States &                                                                                                                                                                                                                                                                                                                                                                                                                                           Input &                                                                      Target \\
\midrule
    0 

/tmp/ipykernel_71327/706422162.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.loc[


In [12]:
df["slots"] = df["states"].map(lambda ex: extract_slots(remove_empty_slots(ex)))

In [13]:
a = df.groupby("split")["slots"].agg(lambda ex: list(set(i for j in ex for i in j))).reset_index()

In [15]:
len(set(i for j in a for i in j))

114

In [2]:
df = pd.concat(
    [load_from_disk(f"../data/processed/multiwoz_{v}")["train"].to_pandas().assign(version=v) for v in ("21", "24")]
)

In [3]:
df = df.drop(columns=["split"]).assign(
    states=lambda df_: df_["states"].map(remove_empty_slots),
    turn_services=lambda df_: df_["turn_services"].map(set),
)

In [4]:
cdf = df.set_index(["dialogue_id", "turn_id", "version"]).unstack(-1)
cdf.columns = [f"{i}_{j}" for i, j in cdf.columns]
cdf = cdf.reset_index()

In [5]:
# turn_services
col = "turn_services"
cdf.loc[cdf[f"{col}_21"] != cdf[f"{col}_24"]]

,dialogue_id,turn_id,sys_utt_21,sys_utt_24,usr_utt_21,usr_utt_24,states_21,states_24,turn_services_21,turn_services_24


In [6]:
# sys_utt
col = "sys_utt"
cdf.loc[cdf[f"{col}_21"] != cdf[f"{col}_24"]]

,dialogue_id,turn_id,sys_utt_21,sys_utt_24,usr_utt_21,usr_utt_24,states_21,states_24,turn_services_21,turn_services_24


In [7]:
# usr_utt
col = "usr_utt"
cdf.loc[cdf[f"{col}_21"] != cdf[f"{col}_24"]]

,dialogue_id,turn_id,sys_utt_21,sys_utt_24,usr_utt_21,usr_utt_24,states_21,states_24,turn_services_21,turn_services_24


In [46]:
# states
sdf = (
    df.assign(
        states=lambda df_: df_["states"].map(
            lambda ex: [(k, i) for k, v in ex.items() for i in v] if ex is not None else [("None",)]
        ),
    )
    .drop(columns=["sys_utt", "usr_utt", "turn_services"])
    .explode(column=["states"])
)

In [60]:
t_21 = sdf.loc[sdf["version"] == "21"]
t_21 = t_21.sort_values(t_21.columns[:-1].tolist()).drop(columns=["version"])

t_24 = sdf.loc[sdf["version"] == "24"]
t_24 = t_24.sort_values(t_24.columns[:-1].tolist()).drop(columns=["version"])

assert t_21.equals(t_24)